In [2]:
import pandas as pd
import re
import os

In [3]:
path='/Users/joel/Desktop/SUTD /T5/SDS/NEW OWN DATA/combined w products.xlsx'
df=pd.read_excel(path)

In [4]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
print(stop_words)

{"doesn't", 'off', 'its', 'was', 'haven', 'y', 'whom', 'this', 's', 'isn', 'our', 'is', 'has', 'any', 'couldn', 'yours', 'from', 'we', 'if', 'yourselves', 'no', 'them', 'up', 'which', 'when', 'themselves', "wouldn't", 'not', 'why', 'into', 're', 'in', "it's", 'i', 'how', 'will', 'yourself', 'being', 'hadn', 'such', 'these', 'under', "isn't", 'have', 'am', 'my', 'be', 'after', 'over', 'won', "wasn't", 'who', 'doing', 'll', 'but', 'don', 'been', "hasn't", "couldn't", 'other', 'the', 'what', 'during', 'more', 'ain', "that'll", 'her', "she's", 'again', 'it', 'here', 'few', 'itself', 'down', 'hasn', 'mightn', "haven't", 'myself', 'your', 'were', "you'll", 'did', 'an', 'to', 'there', 'should', "should've", 'most', 'just', 'himself', "mustn't", 'she', 'before', 'at', 'hers', "aren't", "hadn't", 'm', 'weren', "you've", "you'd", 'all', 'shouldn', 'needn', 'through', 'me', 'only', 'of', 'further', 'above', 'd', 'very', 'they', 'and', 'as', 'ours', 'with', 'having', 'too', 't', 'once', 'ma', 'are

[nltk_data] Downloading package stopwords to /Users/joel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# df = df.dropna(subset=['Q2b', 'Products'])
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
# X= df['Q2b']
# y=df['Products']


In [6]:
print(df.shape)

(935, 9)


In [7]:
# clf = RandomForestClassifier(n_estimators=100, random_state=42)
# clf.fit(X_train, y_train)

# # Step 6: Evaluate the model
# # Evaluate on validation set
# y_val_pred = clf.predict(X_val)
# val_accuracy = accuracy_score(y_val, y_val_pred)
# print(f'Validation Accuracy: {val_accuracy:.2f}')

# # Evaluate on test set
# y_val_pred = clf.predict(X_val)
# test_accuracy = accuracy_score(y_val, y_val_pred)
# print(f'Test Accuracy: {test_accuracy:.2f}')

# # If you have feature importance to display
# import matplotlib.pyplot as plt

# feature_importances = clf.feature_importances_
# features = X.columns
# indices = feature_importances.argsort()

# plt.figure(figsize=(10, 6))
# plt.title('Feature Importances')
# plt.barh(range(len(indices)), feature_importances[indices], align='center')
# plt.yticks(range(len(indices)), [features[i] for i in indices])
# plt.xlabel('Relative Importance')
# plt.show()

In [9]:

import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = df['Q2b']  
y = df['Products'] 

X = X.replace(['NIL', 'n/a', 'na'], np.nan)
df_clean = pd.DataFrame({'Q2b': X, 'Products': y}).dropna(subset=['Q2b', 'Products'])

X = df_clean['Q2b']
y = df_clean['Products']

X = X.astype(str)

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

input_ids = []
attention_masks = []

for text in X:
    encoded_dict = tokenizer.encode_plus(
                        text,
                        add_special_tokens = True,
                        max_length = 64,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   )
    
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(y_encoded)

dataset = TensorDataset(input_ids, attention_masks, labels)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

batch_size = 16

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
val_dataloader = DataLoader(val_dataset, shuffle=False, batch_size=batch_size)

model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = len(label_encoder.classes_),
    output_attentions = False,
    output_hidden_states = False,
)

optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
epochs = 4

for epoch in range(epochs):
    print(f'Epoch {epoch + 1}/{epochs}')
    print('-' * 10)

    total_loss = 0
    model.train()

    for batch in train_dataloader:
        b_input_ids, b_input_mask, b_labels = batch

        model.zero_grad()

        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f'Average training loss: {avg_train_loss:.2f}')

    model.eval()
    predictions, true_labels = [], []

    for batch in val_dataloader:
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        logits = outputs.logits
        predictions.extend(torch.argmax(logits, axis=1).tolist())
        true_labels.extend(b_labels.tolist())

    val_accuracy = accuracy_score(true_labels, predictions)
    print(f'Validation Accuracy: {val_accuracy:.2f}')

model.save_pretrained('fine_tuned_bert')


test_dataloader = DataLoader(val_dataset, shuffle=False, batch_size=batch_size)
model.eval()
predictions, true_labels = [], []

for batch in test_dataloader:
    b_input_ids, b_input_mask, b_labels = batch
    
    with torch.no_grad():
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    logits = outputs.logits
    predictions.extend(torch.argmax(logits, axis=1).tolist())
    true_labels.extend(b_labels.tolist())

test_accuracy = accuracy_score(true_labels, predictions)
print(f'Test Accuracy: {test_accuracy:.2f}')


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/Users/joel/opt/anaconda3/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2760: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/joel/opt/anaconda3/lib/python3.9/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/4
----------
Average training loss: 1.78
Validation Accuracy: 0.62
Epoch 2/4
----------
Average training loss: 1.22
Validation Accuracy: 0.63
Epoch 3/4
----------
Average training loss: 0.97
Validation Accuracy: 0.64
Epoch 4/4
----------
Average training loss: 0.76
Validation Accuracy: 0.73


OSError: [Errno 30] Read-only file system: 'fine_tuned_bert'

In [ ]:
glove_file = '//Users/joel/Downloads/glove/glove.6B.100d.txt' 